In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('E:\\datasets\\splice data.csv',names = ('class','instance name','sequence'))

In [3]:
data = np.random.permutation(data)
data = pd.DataFrame(data)
x = data.iloc[0:,2]
y = data.iloc[0:,0]

In [4]:
x.head()

0                  CCTCGAGAAGGCAAAGCAGACTCTGGAGAACG...
1                 TATGGGGGCACAGGATGAGGAGGAAGGAATCCA...
2               GTGCCCATCACCAACGCCACCCTGGACCGGGTGAG...
3                    TTGAGGAGAAGATCACACACAGTCCCCTGA...
4             TCCCCGTATCTTATCTCTGTCACACTCCAGCTGGTTC...
Name: 2, dtype: object

#converting y to int
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label = LabelEncoder()
y = label.fit_transform(y)

In [5]:
def state(u):
    for n,i in enumerate(u):
        if i == 'IE':
            u[n] = 0
        elif i == 'N':
            u[n] = 1
        elif i == 'EI':
            u[n] = 2

In [6]:
state(y)

In [7]:
y.head()

0    1
1    1
2    2
3    1
4    0
Name: 0, dtype: object

In [8]:
#converting y into matrix
from keras.utils import to_categorical
y = to_categorical(y)

Using TensorFlow backend.


In [9]:
#to remove spaces
x=x.str.strip()

In [10]:
x.head()

0    CCTCGAGAAGGCAAAGCAGACTCTGGAGAACGAGCGGGGGGAGCTG...
1    TATGGGGGCACAGGATGAGGAGGAAGGAATCCAAGACTTGGATGGA...
2    GTGCCCATCACCAACGCCACCCTGGACCGGGTGAGTGCCTGGGCTA...
3    TTGAGGAGAAGATCACACACAGTCCCCTGACCATCTGCTTCCCTGA...
4    TCCCCGTATCTTATCTCTGTCACACTCCAGCTGGTTCCTGCCCTGG...
Name: 2, dtype: object

In [11]:
#to convert into kmers
def getKmers(x, size=6):
    return [x[z:z+size].lower() for z in range(len(x) - size + 1)]

x=x.apply(getKmers)

In [12]:
x

0       [cctcga, ctcgag, tcgaga, cgagaa, gagaag, agaag...
1       [tatggg, atgggg, tggggg, gggggc, ggggca, gggca...
2       [gtgccc, tgccca, gcccat, cccatc, ccatca, catca...
3       [ttgagg, tgagga, gaggag, aggaga, ggagaa, gagaa...
4       [tccccg, ccccgt, cccgta, ccgtat, cgtatc, gtatc...
                              ...                        
3185    [tactct, actctc, ctctcc, tctcct, ctcctt, tcctt...
3186    [atattc, tattcc, attcct, ttccta, tcctaa, cctaa...
3187    [catctg, atctga, tctgag, ctgagg, tgaggt, gaggt...
3188    [cagctg, agctgc, gctgca, ctgcag, tgcaga, gcaga...
3189    [ggcctc, gcctct, cctctc, ctctcc, tctcct, ctcct...
Name: 2, Length: 3190, dtype: object

In [13]:
#converting words into sentences
for i in range(len(x)):
    x[i] = ' '.join(x[i])
x[0]

'cctcga ctcgag tcgaga cgagaa gagaag agaagg gaaggc aaggca aggcaa ggcaaa gcaaag caaagc aaagca aagcag agcaga gcagac cagact agactc gactct actctg ctctgg tctgga ctggag tggaga ggagaa gagaac agaacg gaacga aacgag acgagc cgagcg gagcgg agcggg gcgggg cggggg gggggg ggggga ggggag gggagc ggagct gagctg agctgg gctggc ctggcc tggcca ggccaa gccaac ccaacg caacga aacgag acgagg cgaggt gaggtg aggtga ggtgaa'

In [14]:
#dividing the dataset into training set and test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [15]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras import layers
from keras.layers import Dense,Dropout,Activation, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM


In [16]:
#converting into vectors
tokens = Tokenizer()
tokens.fit_on_texts(x_train)
x_train = tokens.texts_to_sequences(x_train)
x_test = tokens.texts_to_sequences(x_test)

vocab_size = len(tokens.word_index) + 1
print("Total words", vocab_size)

Total words 4157


In [17]:
# padding 
from keras.preprocessing.sequence import pad_sequences
maxlen =55

x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_test = pad_sequences(x_test, padding='post', maxlen=maxlen)


In [18]:
model = Sequential()
model.add(Embedding(vocab_size,30,input_length=maxlen))
model.add(Dropout(0.2))
model.add(Conv1D(10, 5,activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(20,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3,activation='softmax'))
model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 55, 30)            124710    
_________________________________________________________________
dropout_1 (Dropout)          (None, 55, 30)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 51, 10)            1510      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 25, 10)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                5020      
______________________

In [19]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [20]:
history = model.fit(x_train, y_train,validation_split=0.1,epochs=20,verbose=True, batch_size=10)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 2296 samples, validate on 256 samples
Epoch 1/20





2296/2296 [==============================] - 3s 1ms/step - loss: 0.9848 - acc: 0.5436 - val_loss: 0.7427 - val_acc: 0.7461
Epoch 2/20
2296/2296 [==============================] - 1s 649us/step - loss: 0.4298 - acc: 0.8454 - val_loss: 0.2320 - val_acc: 0.9180
Epoch 3/20
2296/2296 [==============================] - 1s 601us/step - loss: 0.1571 - acc: 0.9486 - val_loss: 0.1764 - val_acc: 0.9453
Epoch 4/20
2296/2296 [==============================] - 2s 665us/step - loss: 0.0950 - acc: 0.9665 - val_loss: 0.1590 - val_acc: 0.9453
Epoch 5/20
2296/2296 [==============================] - 2s 690us/step - loss: 0.0764 - acc: 0.9695 - val_loss: 0.1911 - val_acc: 0.9414
Epoch 6/20
2296/2296 [==============================] - 2s 688us/step - loss: 0.0609 - acc: 0.9743 - val_loss: 0.1918 - val_acc: 0.9258
Epoch 7/20
2296/2296 [===============

In [21]:
loss_and_metrics = model.evaluate(x_test, y_test, verbose=1)
print(loss_and_metrics)

638/638 [==============================] - 0s 124us/step
[0.20979163880667248, 0.942006268471386]


In [22]:
labels = [ '0','1','2']

In [23]:
y_pred = model.predict(x_test).round()

In [24]:
y_pred

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [25]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

In [26]:
cm

array([[145,  14,   5],
       [  8, 316,   4],
       [  1,   5, 140]], dtype=int64)